**数据读取与保存**

Spark 支持多种输入输出数据源，一部分原因是 Spark 本身是基于 Hadoop 生态圈而构建，特别是 Spark 可以通过 Hadoop MapReduce 所使用的 InputFormat 和 OutFormat 接口访问数据，大部分常见的文件格式与存储系统（例如 S3、HDFS、Cassandra、HBase等）都支持这种接口。

不过，基于这些原始接口构建出的高层 API 会更常用。本章节会介绍以下三类常见的数据源：

* **文件格式与文件系统**

    对于存储在本地文件系统或分布式文件系统（如 NFS、HDFS、Amazon S3等）中的数据， Spark 可以访问很多种不同的文件格式，包括文本文件、JSON、SequenceFile 以及 Protocal buffer。我们会展示几种常见格式的用法，以及 Spark 针对不同文件系统的配置和压缩选型。


* **Spark SQL 中的结构化数据源**

    之后的章节我们会介绍 Spark SQL 模块，它针对包括 JSON 和 Apache Hive 在内的结构化数据源，为我们提供了一套更加简洁高效的 API。


* **数据库与键值存储**

    我们还会概述 Spark 自带的库和一些第三方库，它们可以用来连接 Cassanda、HBase、Elasticsearch 以及 JDBC 源。

In [1]:
import findspark
# 找到spark的安装目录
spark_path = 'C:\spark232_hadoop27'
findspark.init(spark_path,edit_rc=True)
import pyspark
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('Chen_App')
try:
    sc.stop()
except:
     print('there is none sparkcntext on running')   
sc = SparkContext(conf=conf)

there is none sparkcntext on running


# 文件格式

## 文本文件

在 Spark 中读取文本文件很容易，当我们将一个文本文件读取为 RDD 时，输入的每一行都会成为 RDD 的一个元素。也可以将多个完整的文本文件一次性读取为一个 pair RDD,其中 key 是文件名， value 是文件内容。

***
**读取文本文件**

### textFile()

`textFile(name, minPartitions=None, use_unicode=True)`:Read a text file from HDFS, a local file system (available on all nodes), or any Hadoop-supported file system URI, and return it as an RDD of Strings.

If use_unicode is False, the strings will be kept as str (encoding as utf-8), which is faster and smaller than unicode. (Added in Spark 1.2)

In [2]:
lines = sc.textFile(spark_path + '\README.md')
lines.top(1)

['will run the Pi example locally.']

### wholeTextFiles()

`wholeTextFiles(path, minPartitions=None, use_unicode=True)`:Read a directory of text files from HDFS, a local file system (available on all nodes), or any Hadoop-supported file system URI. Each file is read as a single record and returned in a key-value pair, where the key is the path of each file, the value is the content of each file.

If use_unicode is False, the strings will be kept as str (encoding as utf-8), which is faster and smaller than unicode. (Added in Spark 1.2)

注：使用这个 API 的时候要注意文件足够小才可以。

In [11]:
directory_path = spark_path + '/batchfiles_test_20190422_chenbofeng'
batchFiles = sc.wholeTextFiles(directory_path)
batchFiles.keys().collect()

['file:/C:/spark232_hadoop27/batchfiles_test_20190422_chenbofeng/README_01.md',
 'file:/C:/spark232_hadoop27/batchfiles_test_20190422_chenbofeng/README_02.md']

***
**保存文本文件**

### saveAsTextFile()

`saveAsTextFile(path, compressionCodecClass=None)`:Save this RDD as a text file, using string representations of elements.

Parameters:	
* path – path to text file
* compressionCodecClass – (None by default) string i.e. “org.apache.hadoop.io.compress.GzipCodec”

Spark 将传入的 `path` 作为目录对待(如果是一个存在的路径会报错，所以必须保证路径是新的路径)，会在那个目录下输出多个文件，这样 Spark 就可以在多个节点上并行输出了。

这个方法中，我们无法控制数据的哪一部分输出到哪个文件中，不过有些输出格式支持控制。

In [18]:
save_directory_path = spark_path + '/savefiles_test_20190422_chenbofeng'
try:
    lines.saveAsTextFile(save_directory_path)
except:
    print('directory_path already exist')

directory_path already exist


## JSON

JSON 是一种使用比较广的半结构化数据格式，读取 JSON 数据最简单的方式是将数据作为文本文件读取，然后使用 JSON 解析器来对RDD的值进行映射操作。之后章节还会介绍通过 Spark SQL 读取 JSON 数据。

***
**读取 JSON**

将数据作为文本文件读取，然后对 JSON 数据进行解析

In [19]:
import json
data = lines.map(lambda x: json.loads(x))

注意：处理格式不正确的记录有可能会引起很严重的问题，尤其对于像 JSON 这样的半结构化数据。对于小数据集，可以接受在遇到错误的输入时停止程序，但是对于大规模数据集来说，格式错误是家常便饭。如果选择跳过格式不正确的数据，应该尝试使用累加器来跟踪错误的个数。

***
**保存 JSON**

In [33]:
out_directory_path = spark_path + '/outputfiles_test01_20190422_chenbofeng'
try:
    (data.filter(lambda x:x['Python'])).map(lambda x:json.dumps(x)).saveAsTextFile(out_directory_path)
except:
    print('out_directory_path already exist')

out_directory_path already exist


## CSV 与 TSV

逗号分隔值(CSV)文件每行都有固定数目的字段，字段间用逗号隔开(在制表符分隔值文件，即 TSV 文件中用制表符隔开)。

***
**读取 CSV**

读取 CSV/TSV 数据和读取 JSON 类似，都需要先把文件当作普通文本文件来读取数据，再对数据进行处理。

In [37]:
## 在Python 中使用 textFile() 来读取 CSV
import csv
from io import StringIO

# def loadRecord(line):
# """解析一行 CSV 记录"""
#     input = StringIO.StringIO(line)
#     reader = csv.DictReader(input, filenames=['name', 'favouriteAnimal'])
#     return reader.next()
# input = sc.textFile(inputFile).map(loadRecord)

如果字段中嵌有换行符，就需要完整读入整个文件，然后解析各个字段。如果每个文件都很大，读取和解析的过可能会成为性能瓶颈。

In [38]:
## 在Python 中完整读取 CSV
import csv
from io import StringIO

# def loadRecords(fileNameContents):
#     """读取给定文件中的所有记录"""
#     input = StringIO.StringIO(fileNameContents[1])
#     reader = csv.DictReader(input, filenames=['name', 'favouriteAnimal'])
#     return reader
# input = sc.wholeTextFiles(inputFile).map(loadRecords)

如果只有一小部分输入文件，需要使用 `wholeTextFile()` 方法，可能还需要对输入数据进行重新分区使得 Spark 能够高效地并行化执行后续操作。

***
**保存CSV**

和 JSON数据一样，写出 CSV/TSV 数据相当简单。

In [39]:
## 在Python 中完整读取 CSV
import csv
from io import StringIO

# def writeRecords(records):
#     """写出一些CSV记录"""
#     output = StringIO.StringIO()
#     writer = csv.DictWriter(output, filenames=['name', 'favouriteAnimal'])
#     for record in records:
#         writer.writerow(record)
#     return [output.getvalue()]
# pandaLovers.mapPartitons(writeRecords).saveAsTextFile(outputFile)

## SequenceFile

暂时省略

## 对象文件

暂时省略

## Hadoop 输入输出格式

暂时省略

## 文件压缩

暂时省略

# 文件系统

Spark 支持很多种文件系统，可以使用任何我们想要的文件格式。

## 本地/“常规”文件系统

Spark 支持从本地文件系统中读取文件，不过它**要求文件在集群中所有节点的相同路径下都可以找到。**

一些像 NFS、AFS以及 MapR 的 NFS layer 这样的网络文件系统会把文件以常规文件系统的形式暴露给用户。如果你的数据已经在这些系统中，那么你只需要指定输入为一个 `file://`路径；只要这个文件系统挂载在每个节点的同一路径下， Spark 就会自动处理。

## Amazon S3

暂时省略

## HDFS

Hadoop 分布式文件系统(HDFS) 是一种广泛使用的文件系统， Spark 能够很好地使用它。 HDFS 被设计为可以在廉价的硬件上工作，有弹性地应对节点失败，同时提供高吞吐量。 Spark 和 HDFS 可以部署在同一批机器上，这样 Spark 可以利用数据分布尽量避免一些网络开销。

在 Spark 中使用 HDFS 只需要将输入输出的路径指定为`hdfs://master:port/path`就可以了。

注：HDFS 协议随着 Hadoop 版本改变而改变，所以 Spark 和其所依赖的 Hadoop 版本要和谐。

# Spark SQL 中的结构化数据

Spark SQL 是在 Spark 1.0 加入 Spark 的插件，并快速成为 Spark 中比较受欢迎的操作结构化和半结构化的方式。**结构化数据**是指具有 _**结构信息**_ 的数据——也就是所有的数据记录都具有一致字段结构的集合。

Spark SQL 支持多种结构化数据源作为输入，而且由于 Spark SQL 知道数据的结构信息，它还可以从这些数据源中只读取所需字段。更详细的部分之后的章节会介绍，我们现在只展示如何使用它从一些常见数据源中读取数据。

在各种情况下，我们把一条 SQL 查询给 Spark SQL，让它对一个数据源执行查询，然后得到由 Row 对象组成的 RDD。在 Python 中，可以使用 `row[column_number]` 或者 `row.column_number` 来访问元素。

## Apache Hive

Apache Hive 是 Hadoop 上的一种常见的结构化数据源。 Hive 可以在 HDFS 内或者在其他存储系统上存储多种格式的表。Spark SQL 可以读取 Hive 支持的各种表。

要把 Spark SQL 连接到已有的 Hive 上，需要提供 Hive 的配置文件。需要将 `hive-site.xml` 文件复制到 Spark 的 `./conf/` 目录下。之后，再创建出 `HiveContext` 对象，也就是 Spark SQL 的入口。然后你就可以使用 Hive 查询语言(HQL) 来对你的表进行查询并以行组成的 RDD 的形式拿到返回数据。

**Hadoop 配置还有一些问题没解决，所以Hive 无法正常打开，待修复**

In [41]:
from pyspark.sql import HiveContext

hiveCtx = HiveContext(sc)
# rows = hiveCtx.sql("SELECT name,age FROM users")

## JSON

如果你有 结构一致的 JSON 数据，要读取之前首先需要和使用 Hive 一样创建 HiveContext（不过这种情况下我们不需要安装好 Hive，也就是说不需要 `hive-site.xml` 文件）。然后使用 `HiveContext.jsonFile` 方法来从整个文件中获取由 Row 组成的 RDD。

除了使整个 Row 对象，也可以将 RDD 注册成一张表，然后从中选出特定的字段。假设我们有一个文件 `tweets.json`。 

In [42]:
# tweets = hiveCtx.jsonFile('tweets.json')
# tweets.registerTempTable('tweets')
# result = hiveCtx.sql("select user.name,text from tweets")

# 数据库

## Cassandra

暂时省略

## HBase

暂时省略

## Elasticsearch

暂时省略